# Reinforcement learning


## Modell basiert

### Online vs. Batch

## Modell frei

### Passives lernen vs. aktives lernen


### Adaptive dynamic programming (ADP) 


### Temporal-difference (TD)


### Q-learning

### Monte-Carlo-Methode